There is code for several things in this notebook:
- CLR transform and pseudo-value imputation
- Taxonomic assignment at i.e. genus level
- merge microbiome data with sg90 metadata

In [1]:
import pandas as pd
import numpy as np
import sys
import os
from scipy.stats import gmean

In [2]:
# Get the directory of the current script
script_dir = os.path.dirname(os.path.abspath("Microbiome_prep.ipynb"))
# Build the path to the data directory
data_dir = os.path.join(script_dir, '../../OralMicrobiome_SG90/Full_data/3_feature_profiling/1_feature_tables')
data_dir2 = os.path.join(script_dir, '../../OralMicrobiome_SG90/Full_data/6_taxonomy_community')
# Normalize the path
data_dir = os.path.abspath(data_dir)
data_dir2 = os.path.abspath(data_dir2)
# Build the full path to the file
file_path1 = os.path.join(data_dir, 'feature_table.xlsx')
file_path2 = os.path.join(data_dir2, 'feature_taxonomy_split.xlsx')
file_path3 = os.path.join(data_dir, "feature_table_statistics.xlsx")

In [104]:
df = pd.read_excel(file_path1)

In [105]:
mapping = pd.read_excel(file_path2)

In [106]:
stats = pd.read_excel(file_path3)

In [107]:
# Step 1: Merge 'df' with 'mapping_df' to get 'Genus' for each 'feature ID'
df = df.merge(mapping[['feature ID', 'Genus']], on='feature ID', how='left')

# Step 2: Replace 'feature ID' with 'Genus'
df['feature ID'] = df['Genus']

# Step 3: Drop the 'Genus' column (since it's now in 'feature ID')
df = df.drop(columns=['Genus'])

# Identify numerical columns (excluding 'feature ID')
numerical_cols = df.select_dtypes(include='number').columns.tolist()

# Group by 'feature ID' (now containing 'Genus') and sum numerical columns
df = df.groupby('feature ID')[numerical_cols].sum().reset_index()
df = df.rename(columns={"feature ID":"Genus"})

In [108]:
#just to check no data was lost

# Step 1: Compute the sum of each numerical column
#columns_to_sum = df.select_dtypes(include='number').columns.tolist()
#sums = df[columns_to_sum].sum()

# Step 2: Prepare the row data
#row_data = ['Column Total'] + sums.tolist()

# Step 3: Insert the sums at the first position
#df.loc[-1] = row_data  # Assign to index -1
#df = df.sort_index().reset_index(drop=True)  # Sort the index to bring -1 to the top

In [97]:
# Step 1: Transpose the DataFrame
df_t = df.transpose()

# Step 2: Set the first row as column names
# If the index label of the first row is known (e.g., 'feature ID')
df_t.columns = df_t.loc['Genus']

# Step 3: Drop the first row since it's now the header
df_t = df_t.drop('Genus')

# Step 4: Reset the index to turn the index into a column
df_t = df_t.reset_index()

# Step 5: Rename the index column to something meaningful, e.g., 'Measurement'
df_t = df_t.rename(columns={'index': 'Sample_ID'})

#df_t = df_t.set_index('Sample_ID')

df = df_t

In [98]:
df.to_csv("Genus_Raw.csv", index = False)

In [99]:
# Function to calculate detection limit and impute zeros with pseudo values

def impute_pseudo_values(df):
    for index, row in df.iterrows():
        #print(index)
        # Step 1: Calculate the total sum for the row
        total_sum = row[1:].sum()
        
        if total_sum > 0:
            # Step 2: Calculate the detection limit
            detection_limit = 1 / total_sum
            
            # Step 3: Define the range for the uniform distribution
            lower_bound = 0.1 * detection_limit
            upper_bound = detection_limit

            # Step 4: Apply pseudo values where there are zeros
            df.iloc[index] = row.apply(
                lambda x: np.random.uniform(lower_bound, upper_bound) if x == 0 else x
            )
    
    return df

# Apply the function to your DataFrame
df_imputed = impute_pseudo_values(df)
df = df_imputed

def clr(df):
    
    # Assuming the first column is non-numeric (e.g., sample IDs)
    numeric_cols = df.columns[1:]
    clr_values = []
    for index, row in df.iterrows():
        # Extract numeric values
        numeric_row = row[numeric_cols].astype(float)
        
        # Ensure all values are positive
        if (numeric_row <= 0).any():
            raise ValueError(f"Non-positive values found in row {index}.")
        
        # Compute geometric mean
        geom_mean = gmean(numeric_row)
        
        # Compute CLR-transformed values
        clr_row = np.log(numeric_row / geom_mean)
        
        # Append CLR values with index
        clr_values.append(clr_row)
    
    # Create a DataFrame from CLR values
    clr_df = pd.DataFrame(clr_values, columns=numeric_cols)
    
    # If you want to include the non-numeric identifier column
    clr_df.insert(0, df.columns[0], df.iloc[:, 0])
    
    return clr_df

clr_df = clr(df)
df = clr_df

In [100]:
df.to_csv("Genus_CLR.csv", index = False)

In [101]:
df

Genus,Sample_ID,g__ADurb.Bin063-1,g__Abiotrophia,g__Absconditabacteriales_(SR1)_unclassified,g__Acetatifactor,g__Acetobacter,g__Acholeplasma,g__Acidaminococcus,g__Acidibacter,g__Acidipila,...,g__WPS-2_unclassified,g__WS6_(Dojkabacteria)_unclassified,g__Weeksellaceae_unclassified,g__Weissella,g__Wolinella,g__Xanthobacteraceae_unclassified,g__Xanthomonas,g__Zoogloea,g__env.OPS_17_unclassified,g__unclassified
0,F_80004,-1.672917,14.492515,12.746665,-0.687900,-2.140179,-1.233329,-1.752943,-0.768528,-1.002734,...,-1.194591,-0.700935,-0.647549,-2.738790,-0.718314,-1.066135,-1.552914,-0.869826,-0.879278,-2.161914
1,F_80001,-2.679886,13.705539,-2.928038,-2.856896,-2.560571,-3.539508,-3.421723,-2.729916,-2.930457,...,-3.935528,-2.186815,-2.100233,-2.630003,-3.308113,-2.044435,-2.492038,-2.820255,-2.449954,-2.802513
2,F_80020,-1.675339,11.512263,-2.329797,-2.120262,-3.024316,-1.996212,-1.727647,9.902825,10.819115,...,-2.450707,-2.376341,-3.897041,-3.779175,-3.535187,-1.823895,-2.012793,-2.792838,-2.866250,9.902825
3,M_80007,-0.590243,12.824242,-0.587620,-0.309616,-1.062507,-0.288533,-2.204258,-0.467837,-0.408626,...,-1.964801,-0.586144,-0.449736,-0.874082,-0.477725,-1.687549,-1.159347,-0.763873,-1.061104,-1.311856
4,F_80019,-0.831581,-1.233576,-2.788038,-1.925624,-1.761818,-1.090988,-0.647580,-0.711256,-1.586153,...,-2.385732,-0.585663,-0.716818,-2.647871,-0.955939,-1.449396,-0.629602,-0.826379,-0.640179,10.483597
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
372,M_47170,-1.284733,12.536236,14.237341,-1.433372,-0.924672,-0.671854,-1.866749,-1.551227,-0.514637,...,-1.603772,-0.661770,-0.642861,-0.768255,-2.406814,-0.704617,-1.783489,-0.744895,-2.420726,-1.037746
373,F_5703,-1.468469,13.648284,10.456437,-1.125935,-1.154051,-1.887560,-1.836279,-1.018055,-1.551991,...,-1.410420,-0.924784,-1.043180,-1.055769,-1.597905,-2.606881,-1.299190,-1.241955,-2.380574,10.050972
374,M_67704,-0.170371,-0.591218,-0.118839,-1.230108,-0.096304,-0.885887,-0.100236,-2.077820,-0.319801,...,-1.252864,-0.954627,-1.094514,-2.080670,-0.738796,-1.955839,-0.226799,-2.216457,-0.457239,-1.444750
375,M_14654,-0.892891,-1.965942,-0.401876,-0.921119,-0.739386,-1.476351,-0.343411,-1.183201,-1.167773,...,-0.861427,-0.312389,-0.616735,-0.475453,-0.092659,-0.487046,-0.309247,-0.717444,-0.267937,-1.300483


In [5]:
#prep non-assigned data for rarefaction, alphas/betas
df = pd.read_excel(file_path1)

dft = df.transpose()
# Move the first row to be the new index
dft.columns = dft.iloc[0]  # Set the first row as the index
dft = dft[1:]
dft = dft.reset_index(drop=False)
dft = dft.rename(columns={"index": "Sample_ID"})
dft.index.name = None

SG90 = pd.read_excel("SG90_TimWehnes_26082024.xlsx")
epigenetic_processed = pd.read_csv("Epigenetic_Processed.csv")

#same for the raw genus data
dft["Sample_ID"] = dft["Sample_ID"].astype(str).replace(r'^[A-Za-z]_', '', regex=True)
epigenetic_processed['Sample_ID'] = epigenetic_processed['Sample_ID'].astype(str)

#merge on overlapping samples, sample 80004 has had blood taken twice
dft = pd.merge(dft, epigenetic_processed, on='Sample_ID', suffixes=('', '_epigenetic'))

# Convert Sample_ID, subno, and alt_ID to strings for comparison
ids = dft["Sample_ID"].astype(str)
SG90["subno"] = SG90["subno"].astype(str)
SG90["alt_ID"] = SG90["alt_ID"].dropna().astype(int).astype(str)

# Identify rows in SG90 where either subno or alt_ID matches Sample_ID in df
matching_rows = SG90[(SG90["subno"].isin(ids)) | (SG90["alt_ID"].isin(ids))].copy()

# Create a new 'Sample_ID' column in result_df
matching_rows["Sample_ID"] = matching_rows.apply(
    lambda row: row["subno"] if row["subno"] in ids.values else row["alt_ID"],
    axis=1
)

# Select specific columns by name
specific_columns = matching_rows[
    [ 
        'demogr_sex',
        'demogr_race',
        "ansur_frax_bmi",
        "subs_use_smoke_consolidated",
        "others_Blood_collection_date", 
        "others_Saliva_collection_date",
    ]
]

# Concatenate all the selected columns
result_df = pd.concat([matching_rows[["Sample_ID"]], specific_columns], axis=1)
# Merge into final df
merged_df = pd.merge(dft, result_df, on="Sample_ID", how="left")
#merged_df

# Identify duplicate Sample_ID values in merged_df
duplicate_sample_ids = merged_df[merged_df.duplicated('Sample_ID', keep=False)]

#rows 0,1,3,7 removed bc of likely wrong collection date - not possible that it has been collected twice since measurements are identical
merged_df = merged_df.drop([0, 1, 3, 7])
merged_df = merged_df.reset_index(drop=True)

# Check if blood and saliva have equal dates
# Identify rows where the dates are not equal in merged_df
unequal_dates_rows = merged_df[merged_df['others_Blood_collection_date'] != merged_df['others_Saliva_collection_date']]

# remove the 7rows where the dates are not equal
merged_df = merged_df.drop(unequal_dates_rows.index)
merged_df = merged_df.reset_index(drop=True)

df = merged_df.drop(columns=["Date_of_blood_collection", "others_Saliva_collection_date", "others_Blood_collection_date"])

#use median imputation for BMI and Smoking

# Calculate Q1 (25th percentile) and Q3 (75th percentile)
Q1 = df['ansur_frax_bmi'].quantile(0.25)
Q3 = df['ansur_frax_bmi'].quantile(0.75)

# Calculate IQR
IQR = Q3 - Q1

# Define lower and upper bounds for outliers
lower_bound = Q1 - (100 * IQR)
upper_bound = Q3 + (100 * IQR)

# Find outliers: rows where 'ansur_frax_bmi' is either below the lower bound or above the upper bound
outliers = (df['ansur_frax_bmi'] < lower_bound) | (df['ansur_frax_bmi'] > upper_bound)

# Compute the median of the column
median_value = df['ansur_frax_bmi'].median()

# Replace outliers with the median
df.loc[outliers, 'ansur_frax_bmi'] = median_value

# Replace NaN values with the median as well
df['ansur_frax_bmi'].fillna(median_value, inplace=True)

median_value = df['subs_use_smoke_consolidated'].median()
# Fill NA values with the median
df['subs_use_smoke_consolidated'].fillna(median_value, inplace=True)

df.rename(columns={
    'Age_at_blood_collection': 'Age', 
    'demogr_sex': 'Sex',
    'demogr_race': "Race",
    "ansur_frax_bmi": "BMI",
    "subs_use_smoke_consolidated": "Smoking",
    
}, inplace=True)

# Define your updated columns and group definition
df['Group'] = df['Epigenetic_deviation'].apply(lambda x: 'Worse' if x >= np.median(df["Epigenetic_deviation"]) else 'Better')

# List of columns to remove
to_remove = [
    'Epigenetic_average'
]

# Remove the specified columns from the DataFrame
df = df.drop(columns=to_remove)

df_encoded = pd.get_dummies(df, columns=['Race', 'Sex', 'Smoking'], drop_first=False)  # One-hot encode categorical variables
# Replace True with 1 and False with 0 in the entire DataFrame
df = df_encoded.replace({True: 1, False: 0})
df

,Sample_ID,d0ab2c15400fe710288526c9a33083fb,21b04aaabca0f92a5ba0a0b2eaa31aea,0b0a4bdddb823efeb9984bf862fe81f8,c13359a9f895b1a2b9fcf49ceaea0e8a,0c870b228c94b71c91663a829ef6f119,91c9e0419bfe8038aed98cba9b51cb9d,629676293377721f621917a3f268dbce,91eadffc47a7950a37aae47e5e289b92,e73c0fa69ad4221f822e2e5dcb920e76,...,Group,Race_1.0,Race_2.0,Race_3.0,Race_4.0,Sex_1.0,Sex_2.0,Smoking_0.0,Smoking_1.0,Smoking_2.0
0,80004,391,694,1058,364,624,0,0,0,647,...,Worse,1,0,0,0,0,1,1,0,0
1,80001,2782,3982,2966,1084,5608,0,0,0,840,...,Better,1,0,0,0,0,1,1,0,0
2,80020,754,1524,6077,330,2481,0,161,24,182,...,Worse,1,0,0,0,0,1,0,1,0
3,80007,2547,3013,292,1254,170,0,0,0,411,...,Better,1,0,0,0,1,0,1,0,0
4,80019,243,48,3864,0,0,0,0,0,76,...,Worse,1,0,0,0,0,1,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
306,43327,2258,0,2049,0,0,0,0,0,0,...,Worse,1,0,0,0,0,1,1,0,0
307,47170,703,277,12,3706,12,2830,0,1400,457,...,Worse,1,0,0,0,1,0,0,0,1
308,5703,974,2522,660,1000,51,0,0,39,597,...,Better,1,0,0,0,0,1,1,0,0
309,14654,816,2435,47,0,0,0,0,0,65,...,Better,1,0,0,0,1,0,0,0,1


In [6]:
matching_rows

,subno,redcap_event_name,f4_no,subno_schs,alt_ID,group,status,participant_study_status_complete,demogr_doi,demogr_dob,...,others_v4_interviewer,others_v4_interviewerid,others_visit_4_complete,others_v5_interviewer,others_v5_interviewerid,others_visit_5_complete,others_v5_date,others_v5_sttime,others_v5_entime,Sample_ID
4,1239,baseline_arm_3,70.0,NaN,90133,2,1.0,2,2017-09-01,1929-07-15,...,Chiu Swee Gam,543.0,2.0,Xue Wanqin,21.0,2.0,2017-11-20 00:00:00,12:35:00,14:30:00,90133
7,1977,baseline_arm_3,516.0,NaN,90150,2,1.0,2,2017-08-21,1931-04-16,...,Chong Yu Hing,529.0,2.0,Xue Wanqin,21.0,2.0,2017-11-29 00:00:00,09:00:00,11:40:00,90150
9,2078,baseline_arm_3,69.0,NaN,90189,2,1.0,2,2017-08-29,1930-05-27,...,Chiu Swee Gam,543.0,2.0,Xue Wanqin,21.0,2.0,2017-12-14 00:00:00,08:50:00,10:15:00,90189
11,2624,baseline_arm_3,619.0,NaN,NaN,2,1.0,2,2018-01-11,1931-07-15,...,Chow Lee Lin,523.0,2.0,Xue Wanqin,21.0,2.0,2018-10-23 00:00:00,09:00:00,10:30:00,2624
13,2922,baseline_arm_3,65.0,NaN,90132,2,1.0,2,2017-08-24,1930-07-15,...,Chiu Swee Gam,543.0,2.0,Xue Wanqin,21.0,2.0,2017-11-20 00:00:00,08:45:00,10:25:00,90132
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1151,90313,baseline_arm_1,NaN,NaN,NaN,1,1.0,2,2018-12-11,1932-10-02,...,NaN,NaN,NaN,NaN,NaN,NaN,#NULL!,#NULL!,#NULL!,90313
1152,90314,baseline_arm_1,NaN,NaN,NaN,1,1.0,2,2018-12-10,1930-01-01,...,NaN,NaN,NaN,NaN,NaN,NaN,#NULL!,#NULL!,#NULL!,90314
1153,90317,baseline_arm_1,NaN,NaN,NaN,1,1.0,2,2018-12-14,1931-06-10,...,NaN,NaN,NaN,NaN,NaN,NaN,#NULL!,#NULL!,#NULL!,90317
1154,90318,baseline_arm_1,NaN,NaN,NaN,1,1.0,2,2018-12-18,1933-07-11,...,NaN,NaN,NaN,NaN,NaN,NaN,#NULL!,#NULL!,#NULL!,90318


In [7]:
result_df

,Sample_ID,demogr_sex,demogr_race,ansur_frax_bmi,subs_use_smoke_consolidated,others_Blood_collection_date,others_Saliva_collection_date
4,90133,1,1,21.638905,1.0,2018-01-09 00:00:00,2018-01-09 00:00:00
7,90150,2,1,20.992036,0.0,2018-03-05 00:00:00,2018-03-05 00:00:00
9,90189,1,1,22.236735,0.0,2018-05-31 00:00:00,2018-05-31 00:00:00
11,2624,2,1,27.581100,0.0,2019-08-19 00:00:00,2019-08-19 00:00:00
13,90132,1,1,16.847469,1.0,2018-01-09 00:00:00,2018-01-09 00:00:00
...,...,...,...,...,...,...,...
1151,90313,1,1,22.656250,0.0,2019-01-15 00:00:00,2019-01-15 00:00:00
1152,90314,2,1,20.885250,0.0,2018-12-18 00:00:00,2018-12-18 00:00:00
1153,90317,2,4,20.437045,0.0,2018-12-19 00:00:00,2018-12-19 00:00:00
1154,90318,1,1,22.460937,0.0,2019-01-21 00:00:00,2019-01-21 00:00:00


In [9]:
df["Age"]

0      87.961644
1      87.561644
2      89.000000
3      86.460274
4      89.013699
         ...    
306    86.816438
307    86.923288
308    89.712329
309    89.295890
310    90.295890
Name: Age, Length: 311, dtype: float64

In [138]:
df.to_csv("FINAL_UNASSIGNED_RAW.csv", index=False)

In [ ]:
#prep non-assigned data for rarefaction, alphas/betas
df = pd.read_excel(file_path1)

dft = df.transpose()
# Move the first row to be the new index
dft.columns = dft.iloc[0]  # Set the first row as the index
dft = dft[1:]
dft = dft.reset_index(drop=False)
dft = dft.rename(columns={"index": "Sample_ID"})
dft.index.name = None

SG90 = pd.read_excel("SG90_TimWehnes_26082024.xlsx")
epigenetic_processed = pd.read_csv("Epigenetic_Processed.csv")

#same for the raw genus data
dft["Sample_ID"] = dft["Sample_ID"].astype(str).replace(r'^[A-Za-z]_', '', regex=True)
epigenetic_processed['Sample_ID'] = epigenetic_processed['Sample_ID'].astype(str)

#merge on overlapping samples, sample 80004 has had blood taken twice
dft = pd.merge(dft, epigenetic_processed, on='Sample_ID', suffixes=('', '_epigenetic'))

# Convert Sample_ID, subno, and alt_ID to strings for comparison
ids = dft["Sample_ID"].astype(str)
SG90["subno"] = SG90["subno"].astype(str)
SG90["alt_ID"] = SG90["alt_ID"].dropna().astype(int).astype(str)

# Identify rows in SG90 where either subno or alt_ID matches Sample_ID in df
matching_rows = SG90[(SG90["subno"].isin(ids)) | (SG90["alt_ID"].isin(ids))].copy()

# Create a new 'Sample_ID' column in result_df
matching_rows["Sample_ID"] = matching_rows.apply(
    lambda row: row["subno"] if row["subno"] in ids.values else row["alt_ID"],
    axis=1
)

# Select specific columns by name
specific_columns = matching_rows[
    [ 
        'demogr_sex',
        'demogr_race',
        "ansur_frax_bmi",
        "subs_use_smoke_consolidated",
        "others_Blood_collection_date", 
        "others_Saliva_collection_date",
    ]
]

# Concatenate all the selected columns
result_df = pd.concat([matching_rows[["Sample_ID"]], specific_columns], axis=1)
# Merge into final df
merged_df = pd.merge(dft, result_df, on="Sample_ID", how="left")
#merged_df

# Identify duplicate Sample_ID values in merged_df
duplicate_sample_ids = merged_df[merged_df.duplicated('Sample_ID', keep=False)]

#rows 0,1,3,7 removed bc of likely wrong collection date - not possible that it has been collected twice since measurements are identical
merged_df = merged_df.drop([0, 1, 3, 7])
merged_df = merged_df.reset_index(drop=True)

# Check if blood and saliva have equal dates
# Identify rows where the dates are not equal in merged_df
unequal_dates_rows = merged_df[merged_df['others_Blood_collection_date'] != merged_df['others_Saliva_collection_date']]

# remove the 7rows where the dates are not equal
merged_df = merged_df.drop(unequal_dates_rows.index)
merged_df = merged_df.reset_index(drop=True)

df = merged_df.drop(columns=["Date_of_blood_collection", "others_Saliva_collection_date", "others_Blood_collection_date"])

#use median imputation for BMI and Smoking

# Calculate Q1 (25th percentile) and Q3 (75th percentile)
Q1 = df['ansur_frax_bmi'].quantile(0.25)
Q3 = df['ansur_frax_bmi'].quantile(0.75)

# Calculate IQR
IQR = Q3 - Q1

# Define lower and upper bounds for outliers
lower_bound = Q1 - (100 * IQR)
upper_bound = Q3 + (100 * IQR)

# Find outliers: rows where 'ansur_frax_bmi' is either below the lower bound or above the upper bound
outliers = (df['ansur_frax_bmi'] < lower_bound) | (df['ansur_frax_bmi'] > upper_bound)

# Compute the median of the column
median_value = df['ansur_frax_bmi'].median()

# Replace outliers with the median
df.loc[outliers, 'ansur_frax_bmi'] = median_value

# Replace NaN values with the median as well
df['ansur_frax_bmi'].fillna(median_value, inplace=True)

median_value = df['subs_use_smoke_consolidated'].median()
# Fill NA values with the median
df['subs_use_smoke_consolidated'].fillna(median_value, inplace=True)

df.rename(columns={
    'Age_at_blood_collection': 'Age', 
    'demogr_sex': 'Sex',
    'demogr_race': "Race",
    "ansur_frax_bmi": "BMI",
    "subs_use_smoke_consolidated": "Smoking",
    
}, inplace=True)

# Define your updated columns and group definition
df['Group'] = df['Epigenetic_deviation'].apply(lambda x: 'Worse' if x >= np.median(df["Epigenetic_deviation"]) else 'Better')

# List of columns to remove
to_remove = [
    'Epigenetic_average'
]

# Remove the specified columns from the DataFrame
df = df.drop(columns=to_remove)

df_encoded = pd.get_dummies(df, columns=['Race', 'Sex', 'Smoking'], drop_first=False)  # One-hot encode categorical variables
# Replace True with 1 and False with 0 in the entire DataFrame
df = df_encoded.replace({True: 1, False: 0})
df